# Dimensiones observables de calidad de mercado
---
En este notebook se presenta la metodología de preprocesamiento para una base de datos de la acción de Ecopetrol (ECOPETL). El procedimiento descrito estandariza la información para que esta sea utilizada para calcular distintos parámetros de calidad de mercado. 

Una vez las bases de datos hayan sido inicializadas, esta información puede ser interpretada por los métodos de visualización presentados en el notebook `Visualization.ipynb`

Este notebook está organizado de la siguiente forma:

 1. Manejo de datos
 
 2. Profundidad de mercado
 
 3. Buy-sell
 
 4. Price impact
 
 5. Ejemplo

## 1. Manejo de datos
---
En esta sección se definen dos funciones:

 * `GetDate()` - Retorna un DataFrame que contiene los días para los que tenemos datos
 * `StockPreprocessing()` - Estandariza los encabezados de la base de datos y comienza a calcular algunos de los parámetros de calidad de mercado que se quiere monitorear. Estos son:
  * Precio BID, precio ASK, precio medio y *quoted spread*.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import dateutil.parser
from pandas import DataFrame

import warnings
warnings.filterwarnings('ignore')

import time

import dask.dataframe as dd
from dask import delayed

import os

In [2]:
# Funcion que genera un DataFrame con todos los dias (sin repetir) para los cuales se tienen datos

def GetDate(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    Return:
    ------
    days: DataFrame - DataFrame of all days for which there is data
    '''
    
    days = pd.DatetimeIndex(stockdata.index).normalize()
    days = pd.DataFrame(days)
    days.index = stockdata.index
    days.columns = ['dia']
    
    return days.drop_duplicates(keep='first').dia

In [3]:
# Funcion que inicializa las columnas: 'nombre', 'date_time', 'tipo', 'precio', 'volumen',
#                                      'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'

def StockPreprocessing(stockdata, stock_ticker):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    stock_ticker:
    String - Ticker of the stock we are interested in
    
    
    Return:
    ------
    stockdata:
    DataFrame - Data of stocks with the folloeing initialized columns: 
    nombre', 'date_time', 'tipo', 'precio', 'volumen', 'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'
    '''
    
    stockname = stock_ticker + " CB Equity"
    
    # Se cambian los nombres de las columnas y se elimina lo demas
    stockdata         = stockdata[['name', 'times', 'type', 'value', 'size']]
    stockdata.columns = ['nombre','date_time','tipo','precio','volumen']      
    stockdata         = stockdata[['nombre','date_time','tipo','precio','volumen']]
    
    
    # Seleccionamos los datos segun la accion y el horario que nos interesan
    stockdata        = stockdata.loc[(stockdata["nombre"] == stockname)]
    stockdata.index  = stockdata.date_time
    stockdata        = stockdata.between_time('9:30','15:55')
    stockdata['dia'] = pd.DatetimeIndex(stockdata.date_time).normalize() 
    
    # DataFrame de los dias para los cuales stockdata tiene datos
    days = GetDate(stockdata)
    
    # Lista donde se guardaran los resultados de los calculos diarios
    BA = []
    
    for i in days:
        
        # Sacamos los datos para un unico dia
        stockdailydata = stockdata[stockdata.dia == str(i)]
        
        # Inicializamos columnas de precios BID y ASK
        init_values = stockdailydata.precio.values
        d           = {'BID': init_values, 'ASK': init_values}
        BA_df       = pd.DataFrame(data=d)
        
        # Calculamos los valores de los precios BID y ASK
        bid       = stockdailydata['tipo'] == 'BID'
        ask       = stockdailydata['tipo'] == 'ASK'
        BA_df.BID = np.multiply(bid.values, stockdailydata.precio.values)
        BA_df.ASK = np.multiply(ask.values, stockdailydata.precio.values)
        
        # Para los precios iguales a cero se arrastra el valor anterior
        BA_df['BID'] = BA_df['BID'].replace(to_replace = 0, method = 'ffill').values
        BA_df['ASK'] = BA_df['ASK'].replace(to_replace = 0, method = 'ffill').values
        
        # Donde el BID sea menor o igual al precio ASK se pone un NaN
        BA_df = BA_df.where(BA_df.BID <= BA_df.ASK, np.nan)
        
        # Con los precios BID y ASK se calcula el Mid_Price y el Quoted_Spread
        BA_df['Mid_price']     = 0.5*(BA_df['BID'].values + BA_df['ASK'].values)
        BA_df['Quoted_Spread'] = (BA_df['ASK'].values - BA_df['BID'].values)/(BA_df.Mid_price.values)
        
        # Se guarda todo en el DataFrame BA
        BA.append(BA_df)
    
    # Se unifican los valores para todos los dias
    BA        = pd.concat(BA, axis=0)
    BA.index  = stockdata.index
    stockdata = pd.concat([stockdata, BA], axis=1)
        
    return stockdata

## 2. Profundidad de mercado
---
En esta sección utilizamos la librería Dask para paralelizar el cálculo de los siguientes parámetros: *i)* La profundidad BID; *ii)* La profundidad ASK; *iii)* La profundidad TRADE.

El cálculo de estas cantidades se hace de manera diaria. Para ello, se definen las siguientes funciones:

 * `sep_date()` - Esta función arroja una lista cuyas entradas corresponden a los datos de nuestra base para cada día.
 * `DailyDepth()` - Calcula las profundidades especificadas para cada entrada de `sep_date()`.
 * `StockDepth()` - Encapsula las dos funciones anteriores para calcular los parámetros a partir de todos los datos diarios.

In [4]:
def sep_date(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of a stock    
    
    Return:
    ------
    daily_dfs:
    List - A list whose entries are DataFrames of data in a particular day
    '''
    
    # Obtenemos los dias para los cuales tenemos datos
    days = GetDate(stockdata)
    
    # Creamos la lista donde guardaremos los datos correspondientes a un dia
    daily_dfs = []
    
    # Separamos stockdata por dias
    for i in days:
        daily_dfs.append(stockdata.loc[stockdata["dia"] == i])
        
    return daily_dfs

In [52]:
@delayed
def DailyDepth(stockdailydata):
    '''
    Parameters:
    ------
    stockdailydata:
    DataFrame - Data of a stock for a single day
    
    Return:
    ------
    stockdailydata:
    DataFrame - Data of a stock for a single day with three additional columns corresponding to BID depth, 
                ASK depth and TRADE depth
    '''
    
    for j in range(1, np.shape(stockdailydata)[0] ):
        
        #Tipo BID
        if(stockdailydata.tipo[j]=="BID"):
            stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1]     
            if(stockdailydata.precio[j] == stockdailydata.BID[j]):     
                if(stockdailydata.precio[j] == stockdailydata.BID[j-1]):
                    stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1] + stockdailydata.volumen[j]
                elif(stockdailydata.precio[j] != stockdailydata.BID[j-1]):
                    stockdailydata.BID_depth[j] = stockdailydata.volumen[j]
            elif(stockdailydata.precio[j] != stockdailydata.BID[j]):
                stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1]   
                    
        #Tipo ASK
        elif(stockdailydata.tipo[j]=="ASK"):
            stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1]       
            if(stockdailydata.precio[j] == stockdailydata.ASK[j]):
                if(stockdailydata.precio[j] == stockdailydata.ASK[j-1]):
                    stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1] + stockdailydata.volumen[j]
                elif(stockdailydata.precio[j] != stockdailydata.ASK[j-1]):
                    stockdailydata.ASK_depth[j] = stockdailydata.volumen[j]
            elif(stockdailydata.precio[j] != stockdailydata.ASK[j]):
                stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1]
                
        #Tipo TRADE
        elif(stockdailydata.tipo[j]=="TRADE"):
            if(stockdailydata.precio[j] == stockdailydata.ASK[j]):
                stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1]
                stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1] - stockdailydata.volumen[j]
            elif(stockdailydata.precio[j] == stockdailydata.BID[j]):
                stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1] - stockdailydata.volumen[j]
                stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1]
            else:
                stockdailydata.BID_depth[j] = stockdailydata.BID_depth[j-1]
                stockdailydata.ASK_depth[j] = stockdailydata.ASK_depth[j-1]
                
    
    #-------------------------------------------------------------------------------------------------------
    '''
    # Eliminamos los datos que no tienen sentido
    for k in range(np.shape(stockdailydata)[0]):
        if( stockdailydata.BID_depth[k] < 0):
            stockdailydata.BID_depth[k] = 0
            
        if( stockdailydata.ASK_depth[k] < 0):
            stockdailydata.ASK_depth[k] = 0
            
        # Se calcula la profundidad
        stockdailydata.Depth[k] = stockdailydata.BID_depth[k] + stockdailydata.ASK_depth[k]
        
        # Se calcula la log-profundidad
        if(stockdailydata.ASK_depth[k] != 0 and stockdailydata.BID_depth[k] != 0):
            stockdailydata.log_depth[k] = np.log(stockdailydata.BID_depth[k] * stockdailydata.ASK_depth[k])
            
    # Se quitan los NaN de los datos de profundidad
    for l in range(0, len(stockdailydata.tipo)):   
        if (np.isnan(stockdailydata.Quoted_Spread[l]) == True): 
            stockdailydata.BID_depth[l] = 0
            stockdailydata.ASK_depth[l] = 0
            stockdailydata.Depth[l]     = 0
            stockdailydata.log_depth[l] = 0
    '''
    #-------------------------------------------------------------------------------------------------------
    
    stockdailydata["BID_depth"][stockdailydata["BID_depth"] < 0] = 0
    stockdailydata["ASK_depth"][stockdailydata["ASK_depth"] < 0] = 0
    
    stockdailydata["Depth"] = stockdailydata["BID_depth"] + stockdailydata["ASK_depth"]
    stockdailydata["log_depth"] = np.log(stockdailydata["Depth"])
    stockdailydata[["BID_depth", "ASK_depth", "Depth", "log_depth"]][np.isnan(stockdailydata["Quoted_Spread"].values)] = 0
            
    return stockdailydata

In [48]:
def StockDepth(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of a stock
    
    Return:
    ------
    result_df:
    DataFrame - Results of the depth calculations
    '''
    
    # Creamos columnas para las variables de profundidad
    init_values = np.zeros( np.shape(stockdata)[0] ) #stockdata o stockdailydata - creo que es stockdata
    vol = stockdata.volumen # Con esto comentado me daba diferente a Catalina, vamos a ver ahora como da
    stockdata = stockdata.assign(**{'BID_depth': vol, 'ASK_depth': vol,
                          'Depth': init_values, 'log_depth': init_values})
    
    # Separamos los datos por dia
    daily_df = sep_date(stockdata)
    
    # Creamos la lista donde guardaremos los resultados (sin computar)
    delayed_dfs = []
    
    # Aplicamos la funcion DailyDepth() decorada con un @delay
    for df in daily_df:
        delayed_dfs.append( DailyDepth(df) )
        
    # Generamos un DataFrame de Dask a partir de los resultados (delayed) diarios
    result_df = dd.from_delayed(delayed_dfs)
    
    # Retornamos los resultados computados
    return result_df.compute()

## 3. Buy-sell
---
En esta sección calculamos la dirección de la transacción. Es decir, se calcula si la transacción es iniciada por el comprador o por el vendedor. Para ello, definimos la función `InitiatingParty()`. Este método funciona de la siguiente manera:
 * Se filtran los datos para tener únicamente transacciones tipo TRADE
 * Los precios que se ven se comparan con el precio medio:
  * Si el precio ofrecido es **mayor** que el precio medio, la transacción fue iniciada por un *comprador* ($D=+1$)
  * Si el precio ofrecido es **menor** que el precio medio, la transacción fue iniciada por un *vendedor* ($D=-1$)

In [7]:
def InitiatingParty(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of the stock
    
    Return:
    ------
    x:
    DataFrame - DataFrame of TRADE quotes with the party that initiated the trade
    '''
    
    # Se consideran transacciones tipo TRADE unicamente
    x = stockdata[stockdata.tipo == 'TRADE']
    
    # +1: transaccion iniciada por comprador
    buyer  = x.precio.values > x.Mid_price.values
    
    # -1: transaccion iniciada por vendedor
    seller = x.precio.values < x.Mid_price.values
    
    x['iniciado'] = buyer.astype(int) - seller.astype(int)
    
    # Se eliminan los ceros en caso de que los haya
    x['iniciado'] = x['iniciado'].replace(to_replace = 0, method = 'ffill').values
    
    return x

## 4. Price impact
---

El *price impact* es un parámetro no-observable que cuantifica el efecto que tiene la actividad del mercado sobre el precio de un activo. En este notebook calculamos el impacto en el precio según Kyle, quien argumenta que el volumen de activos transados impacta linealmente su precio.

En este modelo, el cambio del precio de mercado por las órdenes de compra está dado por:

$$\Delta P_{i, t, d} = \beta_{i, d}^1 D_{i, t, d}V_{i, t, d} + \varepsilon_{i, t, d}$$

donde $\Delta P_{i, t, d}$ denota el cambio en el precio del activo $i$, en la transacción $t$ del día $d$. $D_{i, t, d}$ es la dirección de la transacción, es decir si la transacción fue iniciada por un comprador ($D = +1$) o por un vendedor ($D=-1$). 

$V_{i, t, d}$ es el volumen de la orden.

En esta sección calculamos el impacto de precio según Kyle 1985. 
 * `ImpactParameters()` - Calcula los parámetros necesarios para realizar la regresión de impacto de Kyle.
 * `KyleImpactRegression()` - Calcula el valor del impacto al precio.

In [8]:
from datetime import timedelta
from sklearn import linear_model as lm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [9]:
def ImpactParameters(stockdata):
    days = GetDate(stockdata)#.drop_duplicates(keep='first').dia
    res = []
    
    for i in days:
        stockdailydata = stockdata[stockdata.dia == str(i)]
        
        stockdailydata['delta_p']    = stockdailydata['precio'].diff()
        stockdailydata['order_flow'] = stockdailydata.volumen.values * stockdailydata.iniciado.values
        
        res.append(stockdailydata)
        
    res_df = pd.concat(res, axis=0)
    return res_df

In [10]:
def KyleImpactRegression(stockdata):
    
    days = GetDate(stockdata)#.drop_duplicates(keep='first').dia
    res = []
    
    for i in days:
        
        stockdailydata = stockdata[stockdata.dia == str(i)]
        
        x1 = stockdailydata.delta_p.values
        x1 = x1.reshape(-1, 1)
        
        x2 = stockdailydata.order_flow.values
        x2 = sm.add_constant(x2.reshape(-1, 1))
        
        result = sm.OLS(x1, x2, missing='drop').fit()
        
        coef = result.params[1]
        pvalue = result.pvalues[1]
        trades = len(stockdailydata)
        
        temp = [i, coef, pvalue, trades]
        res.append(temp)
        
    #res = pd.DataFrame(res, columns=['day', 'reg_coefficient', 'p_value', 'trades'])
    res = pd.DataFrame(res, columns=['dia', 'coef_regresion', 'p_value', 'trades'])
    res = res.set_index('dia')
    
    return res

## Ejemplo
---
Ahora aplicamos las funciones definidas en las secciones previas:

**1. Manejo de datos**

In [11]:
# Establecemos la ruta del directorio para importar los datos
pathData = os.getcwd() + "/datos completos.csv"

In [12]:
# Importamos los datos
data = %time pd.read_csv(pathData, parse_dates=[2], sep=',', na_values='NA',low_memory=False)

CPU times: user 3.31 s, sys: 299 ms, total: 3.6 s
Wall time: 3.6 s


In [13]:
# Preprocesamos los datos
prep_data = %time StockPreprocessing(data, 'ECOPETL')

CPU times: user 1.05 s, sys: 26.4 ms, total: 1.07 s
Wall time: 1.07 s


**2. Profundidad de mercado**

In [49]:
# Calculamos las profundidaes BID, ASK y TRADE

res = %time StockDepth(test)

CPU times: user 3min 1s, sys: 125 ms, total: 3min 1s
Wall time: 3min 2s


In [51]:
# Miramos los resultados
res.head(12)

,nombre,date_time,tipo,precio,volumen,dia,BID,ASK,Mid_price,Quoted_Spread,BID_depth,ASK_depth,Depth,log_depth
date_time,,,,,,,,,,,,,,
2017-03-08 09:30:00,ECOPETL CB Equity,2017-03-08 09:30:00,ASK,1330.0,203000.0,2017-03-08,0.0,1330.0,665.0,2.000000,203000.0,203000.0,406000.0,0.0
2017-03-08 09:30:00,ECOPETL CB Equity,2017-03-08 09:30:00,ASK,1325.0,104523.0,2017-03-08,0.0,1325.0,662.5,2.000000,203000.0,104523.0,307523.0,0.0
2017-03-08 09:30:00,ECOPETL CB Equity,2017-03-08 09:30:00,ASK,1320.0,15279.0,2017-03-08,0.0,1320.0,660.0,2.000000,203000.0,15279.0,218279.0,0.0
2017-03-08 09:30:03,ECOPETL CB Equity,2017-03-08 09:30:03,BID,1310.0,72866.0,2017-03-08,1310.0,1320.0,1315.0,0.007605,72866.0,15279.0,88145.0,0.0
2017-03-08 09:30:04,ECOPETL CB Equity,2017-03-08 09:30:04,BID,1310.0,122866.0,2017-03-08,1310.0,1320.0,1315.0,0.007605,195732.0,15279.0,211011.0,0.0
2017-03-08 09:30:05,ECOPETL CB Equity,2017-03-08 09:30:05,BID,1310.0,202866.0,2017-03-08,1310.0,1320.0,1315.0,0.007605,398598.0,15279.0,413877.0,0.0
2017-03-08 09:30:12,ECOPETL CB Equity,2017-03-08 09:30:12,BID,1310.0,204866.0,2017-03-08,1310.0,1320.0,1315.0,0.007605,603464.0,15279.0,618743.0,0.0
2017-03-08 09:30:27,ECOPETL CB Equity,2017-03-08 09:30:27,BID,1315.0,80000.0,2017-03-08,1315.0,1320.0,1317.5,0.003795,80000.0,15279.0,95279.0,0.0
2017-03-08 09:30:42,ECOPETL CB Equity,2017-03-08 09:30:42,BID,1315.0,50000.0,2017-03-08,1315.0,1320.0,1317.5,0.003795,130000.0,15279.0,145279.0,0.0


In [16]:
# Guardamos los resultados
res.to_csv("Dask_depth_data.csv")

**3. Buy-sell**

In [17]:
# Encontramos la partida que inicia la transaccion
init_trans = InitiatingParty(prep_data)

In [18]:
# Miramos los resultados
init_trans.head()

,nombre,date_time,tipo,precio,volumen,dia,BID,ASK,Mid_price,Quoted_Spread,iniciado
date_time,,,,,,,,,,,
2017-03-03 09:30:14,ECOPETL CB Equity,2017-03-03 09:30:14,TRADE,1315.0,2000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,1
2017-03-03 09:30:22,ECOPETL CB Equity,2017-03-03 09:30:22,TRADE,1305.0,1000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:30:40,ECOPETL CB Equity,2017-03-03 09:30:40,TRADE,1305.0,1351.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1


**4. Price impact**

In [19]:
# Calculamos los parametros de impacto
impact_params = ImpactParameters(init_trans)

# Miramos los resultados
impact_params.head()

,nombre,date_time,tipo,precio,volumen,dia,BID,ASK,Mid_price,Quoted_Spread,iniciado,delta_p,order_flow
date_time,,,,,,,,,,,,,
2017-03-03 09:30:14,ECOPETL CB Equity,2017-03-03 09:30:14,TRADE,1315.0,2000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,1,NaN,2000.0
2017-03-03 09:30:22,ECOPETL CB Equity,2017-03-03 09:30:22,TRADE,1305.0,1000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,-10.0,-1000.0
2017-03-03 09:30:40,ECOPETL CB Equity,2017-03-03 09:30:40,TRADE,1305.0,1351.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-1351.0
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-500.0
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-500.0


In [19]:
# Calculamos el coeficiente de impacto
kyle_reg = KyleImpactRegression(impact_params)

# Miramos los resultados
kyle_reg.head()

,coef_regresion,p_value,trades
dia,,,
2017-03-03,1.884887e-07,7.937333e-01,176
2017-03-06,7.987430e-06,7.632925e-02,277
2017-03-07,1.599549e-06,2.831874e-01,243
2017-03-08,4.147077e-06,3.137254e-02,377
2017-03-09,1.280608e-05,9.602215e-08,289
